<a href="https://colab.research.google.com/github/kaikecc/Reinforcement-Learning/blob/main/src/notebook/RL_Google_Colab_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Importação e Configuração do 3W Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from pathlib import Path
from multiprocessing.dummy import Pool as ThreadPool
from collections import defaultdict
from natsort import natsorted
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Teste GPU TensorFlow

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Feb 23 13:27:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              39W / 300W |    308MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
1.0579385759999695
GPU (s):
0.06144089700001132
GPU speedup over CPU: 17x


In [ ]:
data_path = Path('/content/drive/MyDrive/Colab Notebooks/dataset')

events_names = {0: 'Normal',
                1: 'Abrupt Increase of BSW',
                2: 'Spurious Closure of DHSV',
                3: 'Severe Slugging',
                4: 'Flow Instability',
                5: 'Rapid Productivity Loss',
                6: 'Quick Restriction in PCK',
                7: 'Scaling in PCK',
                8: 'Hydrate in Production Line'
               }
columns = ['P-PDG',
           'P-TPT',
           'T-TPT',
           'P-MON-CKP',
           'T-JUS-CKP',
           'P-JUS-CKGL',
           'T-JUS-CKGL',
           'QGL',
           'class']
rare_threshold = 0.01

In [ ]:
def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            try:
                class_code = int(class_path.stem)
            except ValueError:
                # Se não for possível converter para int, pule este diretório
                continue

            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path


In [ ]:
real_instances = list(class_and_file_generator(data_path, real=True, simulated=False, drawn=False))
simulated_instances = list(class_and_file_generator(data_path, real=False, simulated=True, drawn=False))
drawn_instances = list(class_and_file_generator(data_path, real=False, simulated=False, drawn=True))

In [ ]:
print(f'Number of real instances: {len(real_instances)}')
print(f'Number of simulated instances: {len(simulated_instances)}')
print(f'Number of drawn instances: {len(drawn_instances)}')

Number of real instances: 1019
Number of simulated instances: 939
Number of drawn instances: 20


In [ ]:
def load_instance(instances):
    class_code, instance_path = instances
    try:
        well, instance_id = instance_path.stem.split('_')
        df = pd.read_csv(instance_path, index_col='timestamp', parse_dates=['timestamp'])
        assert (df.columns == columns).all(), "invalid columns in the file {}: {}".format(str(instance_path), str(df.columns.tolist()))
        df['label'] = class_code
        df['well'] = well
        df['id'] = instance_id
        df = df[['label', 'well', 'id'] + columns]
        return df
    except Exception as e:
        raise Exception('error reading file {}: {}'.format(instance_path, e))


df = load_instance(real_instances[0])
df.head()

,label,well,id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class
timestamp,,,,,,,,,,,,
2014-08-06 06:01:41,4,WELL-00004,20140806060141,0.0,17763590.0,106.4911,9559227.0,68.04160,NaN,NaN,NaN,4
2014-08-06 06:01:42,4,WELL-00004,20140806060141,0.0,17764010.0,106.4911,9560363.0,68.04454,NaN,NaN,NaN,4
2014-08-06 06:01:43,4,WELL-00004,20140806060141,0.0,17764430.0,106.4911,9561499.0,68.04749,NaN,NaN,NaN,4
2014-08-06 06:01:44,4,WELL-00004,20140806060141,0.0,17764850.0,106.4911,9562636.0,68.05044,NaN,NaN,NaN,4
2014-08-06 06:01:45,4,WELL-00004,20140806060141,0.0,17765260.0,106.4911,9563771.0,68.05338,NaN,NaN,NaN,4


In [ ]:
import pandas as pd
from natsort import natsorted

# Supondo que real_instances, simulated_instances, drawn_instances e events_names já estejam definidos
instances_class = [{'TYPE OF EVENT': str(c) + ' - ' + events_names[c], 'SOURCE': 'REAL'} for c, p in real_instances] + \
                  [{'TYPE OF EVENT': str(c) + ' - ' + events_names[c], 'SOURCE': 'SIMULATED'} for c, p in simulated_instances] + \
                  [{'TYPE OF EVENT': str(c) + ' - ' + events_names[c], 'SOURCE': 'DRAWN'} for c, p in drawn_instances]
df_class = pd.DataFrame(instances_class)

# Correção aqui: pivot(index, columns, values) com argumentos nomeados
df_class_count = df_class.groupby(['TYPE OF EVENT', 'SOURCE']).size().reset_index(name='count')
df_class_count = df_class_count.pivot(index='TYPE OF EVENT', columns='SOURCE', values='count').fillna(0).astype(int)

# Transposição não é necessária após a correção, mas ajustaremos a orientação conforme sua intenção original
#df_class_count = df_class_count.T

# Ordenação natural dos índices, se necessário
df_class_count = df_class_count.loc[natsorted(df_class_count.index)]

# Reordenar colunas se 'REAL', 'SIMULATED', 'DRAWN' estiverem presentes, adicionando a coluna 'TOTAL'
df_class_count['TOTAL'] = df_class_count.sum(axis=1)
df_class_count.loc['TOTAL'] = df_class_count.sum(axis=0)

# Exibindo o DataFrame final
df_class_count


SOURCE,DRAWN,REAL,SIMULATED,TOTAL
TYPE OF EVENT,,,,
0 - Normal,0,594,0,594
1 - Abrupt Increase of BSW,10,5,114,129
2 - Spurious Closure of DHSV,0,22,16,38
3 - Severe Slugging,0,32,74,106
4 - Flow Instability,0,344,0,344
5 - Rapid Productivity Loss,0,11,439,450
6 - Quick Restriction in PCK,0,6,215,221
7 - Scaling in PCK,10,5,0,15
8 - Hydrate in Production Line,0,0,81,81


# Criação de um único dataframe com todos os poços reais

In [ ]:

# Carregar todos os DataFrames dos arquivos de instâncias reais
df_all_instances_real = [pd.DataFrame(load_instance(instance)) for instance in real_instances]

# Concatenar todos os DataFrames na lista para formar um único DataFrame
df = pd.concat(df_all_instances_real, ignore_index=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14516197 entries, 0 to 14516196
Data columns (total 12 columns):
 #   Column      Dtype  
---  ------      -----  
 0   label       int64  
 1   well        object 
 2   id          object 
 3   P-PDG       float64
 4   P-TPT       float64
 5   T-TPT       float64
 6   P-MON-CKP   float64
 7   T-JUS-CKP   float64
 8   P-JUS-CKGL  float64
 9   T-JUS-CKGL  float64
 10  QGL         float64
 11  class       float64
dtypes: float64(9), int64(1), object(2)
memory usage: 1.3+ GB


# Criação de um dataframe para o desenvolvimento de um Ambiente Python (TF)

Critério de seleção dos poços

1. Com mais de três tipos de eventos: 'WELL-00001', 'WELL-00002', 'WELL-00004', 'WELL-00006'
2. Dois poços para equilibrar a falta de eventos: 'WELL-00015', 'WELL-00016'

In [ ]:
# Lista de poços para treinamento do modelo
wells_to_include = ['WELL-00001', 'WELL-00002', 'WELL-00004', 'WELL-00006', 'WELL-00015', 'WELL-00016']

# Selecionando colunas específicas
columns_to_select = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'class']

# Filtrando o DataFrame para incluir apenas os poços desejados e colunas específicas, e removendo linhas com NaNs
df_env = df[df['well'].isin(wells_to_include)][columns_to_select].dropna()

# Mostrando informações do DataFrame filtrado
df_env.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9645620 entries, 0 to 14516196
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   P-PDG      float64
 1   P-TPT      float64
 2   T-TPT      float64
 3   P-MON-CKP  float64
 4   T-JUS-CKP  float64
 5   class      float64
dtypes: float64(6)
memory usage: 515.1 MB


In [ ]:
nan_count = df_env.isna().sum()

print(nan_count)

P-PDG        0
P-TPT        0
T-TPT        0
P-MON-CKP    0
T-JUS-CKP    0
class        0
dtype: int64


# Pré-processamento dos dados do dataframe



In [ ]:
# Colunas para normalizar, exceto 'class'
columns_to_normalize = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP']

# Aplicando Z-score Standardization
df_env[columns_to_normalize] = (df_env[columns_to_normalize] - df_env[columns_to_normalize].mean()) / df_env[columns_to_normalize].std()

# Converte a coluna 'class' para valor int
df_env['class'] = df_env['class'].astype(int)

# Verifique as primeiras linhas para confirmar a normalização
df_env.head()


,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,class
0,0.195413,-0.056686,-1.411703,1.209522,-0.458901,4
1,0.195413,-0.056684,-1.411703,1.209818,-0.458802,4
2,0.195413,-0.056682,-1.411703,1.210113,-0.458704,4
3,0.195413,-0.056680,-1.411703,1.210408,-0.458605,4
4,0.195413,-0.056678,-1.411703,1.210703,-0.458506,4


# Classe 3W - Ambiente TF

In [ ]:
!pip install tf-agents
!pip install dm-reverb[tensorflow]
!pip install dm-reverb


In [ ]:
import numpy as np
import pandas as pd
from tf_agents.environments import py_environment
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts

class env3W(py_environment.PyEnvironment):
    '''
    Essa classe é um Modelo-Baseado em Ambiente para o problema de detecção de falhas em poços de petróleo.

     1. O ambiente é um repositório di github https://github.com/petrobras/3W
     2. O ambiente é composto por dados de seis poços de petróleo, com cinco variáveis (observações) de entrada ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP'] e um rótulo indentificador de falha [class]
     3. O ambiente é um ambiente de simulação, onde o agente pode escolher entre duas ações: 0 - Não Detectado ou 1 - Detectado para cada observação
     4. A recompensa é calculada com base na ação escolhida e no rótulo de falha [class]:

        Estados dos sensores:
        - rótulo de falha: 0 - Estado Normal
        - rótulo de falha: 1 a 8 - Estável de Anomalia (Falha)
        - rótulo de falha: 101 a 108 - Transiente de Anomalia (Falha)

        Ações/Recompensas:
        - Se o rótulo de falha for 0, a recompensa é 1 se a ação for 0, caso contrário, a recompensa é -100
        - Se o rótulo de falha estiver entre 1 e 8, a recompensa é -100 se a ação for 0, caso contrário, a recompensa é 100
        - Se o rótulo de falha estiver entre 101 e 108, a recompensa é -10 se a ação for 0, caso contrário, a recompensa é 10

    '''
    def __init__(self, dataframe):
        super(env3W, self).__init__()
        self._dataframe = dataframe
        self._index = 0
        # Ação: 0 - Não Detectado ou 1 - Detectado
        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
        num_features = len(['P-PDG', 'P-TPTP', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP'])
        self._observation_spec = array_spec.BoundedArraySpec(shape=(num_features,), dtype=np.float32, name='observation')
        self._state = None
        self._episode_ended = False

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _update_state(self):
        # Seleciona diretamente as colunas necessárias do dataframe
        columns_needed = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP']
        row = self._dataframe.iloc[self._index][columns_needed]
        self._state = row.values

    def _reset(self):
        self._index = 0
        self._update_state()
        self._episode_ended = False
        return ts.restart(np.array(self._state, dtype=np.float32))

    def _step(self, action):
        if self._episode_ended:
            # Isso deve retornar uma chamada para reset(), não a terminação diretamente
            return self.reset()

        if self._index >= len(self._dataframe) - 1:
            # Quando chegar na última linha, marque o episódio como terminado e pare.
            self._episode_ended = True
            reward = self._calculate_reward(action)
            return ts.termination(np.array(self._state, dtype=np.float32), reward=reward)

        # Lógica para processar a ação e atualizar o estado aqui
        self._index += 1
        self._update_state()
        reward = self._calculate_reward(action)
        if self._episode_ended:
            return ts.termination(np.array(self._state, dtype=np.float32), reward=reward)
        else:
            return ts.transition(np.array(self._state, dtype=np.float32), reward=reward, discount=1.0)





    def _calculate_reward(self, action):
        class_value = self._dataframe.iloc[self._index]['class']
        if class_value == 0:
            return 1 if action == 0 else -100
        elif class_value in range(1, 9):
            return -100 if action == 0 else 100
        elif class_value in range(101, 109):
            return -10 if action == 0 else 10
        else:
            # Define uma recompensa padrão para qualquer outro valor de class não especificado
            return 0





# Setup

In [ ]:
#import sys
#import os
#sys.path.append(os.path.join('..'))
#from classes._env3W import env3W  # Ajuste para o caminho correto da sua classe de ambiente

import reverb
import tensorflow as tf


from tf_agents.environments import utils
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.drivers import dynamic_step_driver
from tf_agents.trajectories import trajectory
from tf_agents.utils import common
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.utils import common
from tf_agents.specs import tensor_spec
from tf_agents.networks import sequential
from tf_agents.drivers import py_driver
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils

# Hiperparâmentros

In [ ]:
# Configurações iniciais
num_iterations = 20000  # ou mais, dependendo da complexidade do ambiente

initial_collect_steps = 1000  # Número de passos de coleta inicial
collect_steps_per_iteration = 1  # Passos de coleta por iteração de treinamento
replay_buffer_max_length = 100000  # Tamanho máximo do replay buffer

log_interval = 1000  # Log do progresso a cada 1000 iterações de treinamento
eval_interval = 5000  # Avaliação do desempenho do agente a cada 5000 iterações
batch_size = 64  # Exemplo de tamanho de batch para treinamento
num_iterations = 10000  # Exemplo de número de iterações de treinamento
learning_rate= 1e-3 # Taxa de aprendizado

num_eval_episodes = 2  # @param {type:"integer"}

# Ambiente

No Aprendizado por Reforço (RL), um ambiente representa a tarefa ou problema a ser resolvido. Ambientes padrão podem ser criados em TF-Agents usando conjuntos tf_agents.environments.

1. O ambiente é um repositório di github https://github.com/petrobras/3W
2. O ambiente é composto por dados de seis poços de petróleo, com cinco variáveis (observações) de entrada ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP'] e um rótulo indentificador de falha [class]
3. O ambiente é um ambiente de simulação, onde o agente pode escolher entre duas ações: 0 - Não Detectado ou 1 - Detectado para cada observação
4. A recompensa é calculada com base na ação escolhida e no rótulo de falha [class]:

Estados:
- rótulo de falha: 0 - Estado Normal
- rótulo de falha: 1 a 8 - Estável de Anomalia (Falha)
- rótulo de falha: 101 a 108 - Transiente de Anomalia (Falha)

Ações/Recompensas:
- Se o rótulo de falha for 0, a recompensa é 1 se a ação for 0, caso contrário, a recompensa é -100
- Se o rótulo de falha estiver entre 1 e 8, a recompensa é -100 se a ação for 0, caso contrário, a recompensa é 100
- Se o rótulo de falha estiver entre 101 e 108, a recompensa é -10 se a ação for 0, caso contrário, a recompensa é 10

In [ ]:
#amostra aleatória de 20% dos dados
df_test = df_env.sample(frac=0.2, random_state=42).reset_index(drop=True)


# Criação do ambiente de teste
env = env3W(df_test)


In [ ]:
df_test.head()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,class
0,0.195413,-0.101455,0.037760,-0.805689,-0.349975,0
1,0.195413,-0.098600,0.153103,-0.876860,-0.212660,0
2,0.195413,-0.099491,0.049698,-0.884553,-0.246885,0
3,0.195413,-0.099206,0.127501,-0.902354,-0.236087,0
4,0.195413,-0.063936,-1.401848,0.818197,-0.448728,4


Para validar o ambiente, é usado uma política aleatória para gerar ações e faremos a iteração em mais de 5 episódios para garantir que as coisas estejam funcionando como pretendido. Um erro é gerado se recebermos um time_step que não segue as especificações do ambiente.

In [ ]:
# depois de 1h30 - cancelando verificação
utils.validate_py_environment(env, episodes=5)

In [ ]:
# Testa o método reset para iniciar o ambiente
time_step = env.reset()
print("Estado Inicial:", time_step)

# Executa algumas ações para testar a resposta do ambiente
for _ in range(5):
    action = np.random.randint(0, 2)  # Escolhe uma ação aleatória, 0 ou 1
    time_step = env.step(action)
    print("Após a ação:", action, "Time Step:", time_step)

Estado Inicial: TimeStep(
{'step_type': array(0, dtype=int32),
 'reward': array(0., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([ 0.19541295, -0.10145459,  0.03776014, -0.805689  , -0.34997538],
      dtype=float32)})
Após a ação: 1 Time Step: TimeStep(
{'step_type': array(1, dtype=int32),
 'reward': array(-100., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([ 0.19541295, -0.09860043,  0.15310277, -0.87685996, -0.21266001],
      dtype=float32)})
Após a ação: 0 Time Step: TimeStep(
{'step_type': array(1, dtype=int32),
 'reward': array(1., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([ 0.19541295, -0.09949141,  0.04969804, -0.88455325, -0.24688512],
      dtype=float32)})
Após a ação: 1 Time Step: TimeStep(
{'step_type': array(1, dtype=int32),
 'reward': array(-100., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([ 0.19541295, -0.09920649,  0.12750119, -0.9023537

In [ ]:
df_test['class'].unique()

array([  0,   4, 107, 101,   3,   1,   6,   5, 105,   7, 106, 102,   2])

In [ ]:
env.reset()

TimeStep(
{'step_type': array(0, dtype=int32),
 'reward': array(0., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([ 0.19541295, -0.10145459,  0.03776014, -0.805689  , -0.34997538],
      dtype=float32)})

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)

Observation Spec:
BoundedArraySpec(shape=(5,), dtype=dtype('float32'), name='observation', minimum=-3.4028234663852886e+38, maximum=3.4028234663852886e+38)


In [ ]:
print('Reward Spec:')
print(env.time_step_spec().reward)

Reward Spec:
ArraySpec(shape=(), dtype=dtype('float32'), name='reward')


In [ ]:
print('Action Spec:')
print(env.action_spec())

Action Spec:
BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=1)


No ambiente env3W:

* Observação é um array de cinco floats:

    * Três sensores de pressão: 'P-PDG', 'P-TPT', 'P-MON-CKP'
    * Dois sensores de temperatura: 'T-TPT', T-JUS-CKP'

* A recompensa é um escalar inteiro.

* A ação é um escalar inteiro com duas possibilidades:

    * 0 — "Não Detectado"
    * 1 — "Detectado"

In [ ]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

Time step:
TimeStep(
{'step_type': array(0, dtype=int32),
 'reward': array(0., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([ 0.19541295, -0.10145459,  0.03776014, -0.805689  , -0.34997538],
      dtype=float32)})
Next time step:
TimeStep(
{'step_type': array(1, dtype=int32),
 'reward': array(-100., dtype=float32),
 'discount': array(1., dtype=float32),
 'observation': array([ 0.19541295, -0.09860043,  0.15310277, -0.87685996, -0.21266001],
      dtype=float32)})


Normalmente dois ambientes são instanciados: um para treinamento e outro para avaliação.

In [ ]:
train_env = tf_py_environment.TFPyEnvironment(env)
eval_env = tf_py_environment.TFPyEnvironment(env)  # Avaliação pode ser em um env similar ou diferente


# Agente

O algoritmo utilizado para resolver um problema de RL é representado por um Agente. TF-Agents fornece implementações padrão de uma variedade de Agentes, incluindo:

* DQN (Usado para teste)
* REINFORCE
* DDPG
* TD3
* PPO
* SAC

O agente DQN pode ser utilizado em qualquer ambiente que possua um espaço de ação discreto.

No coração de um Agente DQN está uma QNetwork, um modelo de rede neural que pode aprender a prever QValues (retornos esperados) para todas as ações, dada uma observação do ambiente.

Usaremos tf_agents.networks. para criar uma QNetwork. A rede será composta por uma sequência de camadas tf.keras.layers.Dense, onde a camada final terá 1 saída para cada ação possível.

In [ ]:
fc_layer_params = (100, 50)  # Exemplo de tamanho das camadas totalmente conectadas
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Defina uma função auxiliar para criar camadas densas configuradas com o direito
# ativação e inicializador do kernel.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))


# QNetwork consiste em uma sequência de camadas densas seguidas por uma camada densa
# com unidades `num_actions` para gerar um q_value por ação disponível como sua saída.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))

Agora use tf_agents.agents.dqn.dqn_agent para instanciar um DqnAgent. Além de time_step_spec, action_spec e QNetwork, o construtor do agente também requer um otimizador (neste caso, AdamOptimizer), uma função de perda e um contador de passos inteiro.

In [ ]:

q_net = sequential.Sequential(dense_layers + [q_values_layer])



In [ ]:


optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

# Política

Uma política define a forma como um agente atua em um ambiente. Normalmente, o objetivo da aprendizagem por reforço é treinar o modelo subjacente até que a política produza o resultado desejado.

* O resultado desejado é a identificar uma falha com base nas observações
* A política retorna uma ação (Não Detectado ou Detectado) para cada observação time_step.

Os agentes contêm duas políticas:

* agent.policy — A política principal usada para avaliação e implantação.
* agent.collect_policy — Uma segunda política usada para coleta de dados.

In [ ]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

As políticas podem ser criadas independentemente dos agentes. Por exemplo, use tf_agents.policies.random_tf_policy para criar uma política que selecionará aleatoriamente uma ação para cada time_step.

In [ ]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

Para obter uma ação de uma política, chame o método policy.action(time_step). O time_step contém a observação do ambiente. Este método retorna um PolicyStep, que é uma tupla nomeada com três componentes:

* ação - a ação a ser executada (neste caso, 0 ou 1)
* estado – usado para políticas com estado (isto é, baseadas em ANN)
* info — dados auxiliares, como log de probabilidades de ações

In [ ]:
from tf_agents.trajectories import time_step as ts

time_step = env.reset()

In [ ]:
from tf_agents.policies import random_tf_policy

# Exemplo de criação de uma política aleatória
# Supondo que env seja seu ambiente TF-Agents
random_policy = random_tf_policy.RandomTFPolicy(action_spec=env.action_spec(),
                                                 time_step_spec=env.time_step_spec())


# Métricas e Avaliação

A métrica mais comum usada para avaliar uma política é o retorno médio. O retorno é a soma das recompensas obtidas durante a execução de uma política em um ambiente durante um episódio. Vários episódios são executados, criando um retorno médio.

A função a seguir calcula o retorno médio de uma política, dada a política, o ambiente e um número de episódios

In [ ]:


def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

A execução desse cálculo em random_policy mostra um desempenho de linha de base no ambiente.

In [ ]:
compute_avg_return(eval_env, random_policy, num_eval_episodes)


KeyboardInterrupt: 

# Replay Buffer

Para acompanhar os dados coletados do ambiente, usaremos o Reverb, um sistema de replay eficiente, extensível e fácil de usar da Deepmind. Ele armazena dados de experiência quando coletamos trajetórias e são consumidos durante o treinamento.

Este buffer de reprodução é construído usando especificações que descrevem os tensores que devem ser armazenados, que podem ser obtidos do agente usando agent.collect_data_spec.

In [ ]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

Para a maioria dos agentes, collect_data_spec é uma tupla nomeada chamada Trajetória, contendo especificações para observações, ações, recompensas e outros itens.

In [ ]:
agent.collect_data_spec


In [ ]:
agent.collect_data_spec._fields


# Coleção de dados

Agora execute a política aleatória no ambiente por algumas etapas, registrando os dados no buffer de reprodução.

Aqui estamos usando 'PyDriver' para executar o ciclo de coleta de experiência. Você pode aprender mais sobre o driver TF Agents em nosso

In [ ]:

#@test {"skip": true}
py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(env.reset())

O buffer de repetição agora é uma coleção de trajetórias

In [ ]:
# For the curious:
# Uncomment to peel one of these off and inspect it.
# iter(replay_buffer.as_dataset()).next()


O agente precisa de acesso ao buffer de reprodução. Isso é fornecido pela criação de um pipeline tf.data.Dataset iterável que alimentará os dados para o agente.

Cada linha do buffer de reprodução armazena apenas uma única etapa de observação. Mas como o Agente DQN precisa da observação atual e da próxima para calcular a perda, o pipeline do conjunto de dados irá amostrar duas linhas adjacentes para cada item no lote (num_steps=2).

Este conjunto de dados também é otimizado executando chamadas paralelas e pré-busca de dados.

In [ ]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

dataset

In [ ]:

iterator = iter(dataset)
print(iterator)

# Treinando o agente

Duas coisas devem acontecer durante o ciclo de treinamento:

* coletar dados do ambiente
* usar esses dados para treinar a(s) rede(s) neural(is) do agente

Este exemplo também avalia periodicamente a política e imprime a pontuação atual.

O seguinte levará cerca de 5 minutos para ser executado.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Reset the environment.
time_step = env.reset()

# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

for _ in range(num_iterations):

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

# Visualização

In [ ]:
iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=250)